In [ ]:
import tacman

In [ ]:
import numpy as np
import scipy
import scipy.stats
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt, cm
%matplotlib inline

import sklearn
from sklearn import linear_model
from breze.learn.mlp import Mlp as learn_Mlp

import os

In [ ]:
sensor = 'icub'
datasetpath = '/home/<username>/Datasets/tacman'

In [ ]:
use_shore = True
if use_shore:
    surface = tacman.datasets.shore.raw(path=datasetpath,sensor=sensor)
else:
    surface = tacman.datasets.surface.raw(path=datasetpath,sensor=sensor)

In [ ]:
# split by experiments, not samples
# not including experiment 0,66,68 and  one for iCub (extreme outlier)
if os.path.isfile(sensor + '_' +('shore' if use_shore else 'surface') + '_ind.npy'):
    ind = np.load(sensor + '_' +('shore' if use_shore else 'surface') + '_ind.npy')
else:
    ind = np.arange(0,len(surface.index.levels[0]))
    ind = np.delete(ind, [0,66,68])
    np.random.shuffle(ind)
    np.save(sensor + '_' +('shore' if use_shore else 'surface') + '_ind.npy', ind)
split = 0.7

In [ ]:
train_surface = surface.T[ind[0]].T
for i in ind[1:int(split*len(ind))]:
    train_surface = pd.concat([train_surface, surface.T[i].T])
test_surface = surface.T[ind[int(split*len(ind))]].T
for i in ind[int(split*len(ind))+1:]:
    test_surface = pd.concat([test_surface, surface.T[i].T])

In [ ]:
unit_roll = (25.0/182.6 * 1.8)*13
unit_pitch = 1.8

In [ ]:
X = np.array(train_surface.T['E1':'E19' if sensor == 'biotac' else 'E12'].T)
TX = np.array(test_surface.T['E1':'E19' if sensor == 'biotac' else 'E12'].T)

X = scipy.ndimage.filters.gaussian_filter1d(X, 5, axis=0)
TX = scipy.ndimage.filters.gaussian_filter1d(TX, 5, axis=0)

Yp = np.array(train_surface['pitch'])
TYp = np.array(test_surface['pitch'])

Yr = np.array(train_surface['roll'])
TYr = np.array(test_surface['roll'])

if use_shore:
    Ys = np.array(train_surface['shore'])
    TYs = np.array(test_surface['shore'])

In [ ]:
Yr = Yr
TYr = TYr

In [ ]:
Yp = Yp
TYp = TYp

### Normalisation

In [ ]:
mean = X.mean(0)
X -= mean
std = X.std()
X /= std
TX -= mean
TX /= std

In [ ]:
plt.plot(X[:]);

### Variational Autoencoder

In [ ]:
from breze.learn import sgvb
import climin
import climin.stops

import theano
import theano.tensor as T

from breze.arch.construct.layer.distributions import DiagGauss, NormalGauss
from breze.arch.construct.neural.distributions import MlpDiagGauss
from breze.arch.construct.neural import Mlp

class MlpDiagConstVarGauss(DiagGauss):
    def __init__(self, inpt, n_inpt, n_hiddens, n_output,
                 hidden_transfers, out_transfer_mean='identity',
                 declare=None, name=None, rng=None):
        self.inpt = inpt
        self.n_inpt = n_inpt
        self.n_hiddens = n_hiddens
        self.n_output = n_output
        self.hidden_transfers = hidden_transfers
        self.out_transfer_mean = out_transfer_mean
        self.mean_mlp = Mlp(
            self.inpt, self.n_inpt, self.n_hiddens, self.n_output,
            self.hidden_transfers,
            self.out_transfer_mean,
            declare=declare)
        self.std = declare((1, n_output))
        super(MlpDiagConstVarGauss, self).__init__(
            self.mean_mlp.output,
            self.std**2 + 1e-5)
            
            
class MlpGaussConstVarVisibleVAEMixin(object):
    def make_gen(self, latent_sample):
        return MlpDiagConstVarGauss(
            latent_sample, self.n_latent,
            self.n_hiddens_gen,
            self.n_inpt,
            self.gen_transfers,
            declare=self.parameters.declare)
    
class MlpGaussLatentVAEMixin(object):

    def make_prior(self, sample):
        return NormalGauss(sample.shape)

    def make_recog(self, inpt):
        return MlpDiagGauss(
            inpt, self.n_inpt,
            self.n_hiddens_recog,
            self.n_latent,
            self.recog_transfers,
            out_transfer_mean='identity',
            out_transfer_var= T.exp,
            declare=self.parameters.declare)

class MyVae(sgvb.VariationalAutoEncoder,
                    MlpGaussLatentVAEMixin,
                    MlpGaussConstVarVisibleVAEMixin):
    pass

optimizer = 'rmsprop', {'step_rate': 0.001}
batch_size = 200

n_latent = 128

m = MyVae( int(X.shape[1]),
                    [512] * 2, n_latent, [512] * 2,
                    ['rectifier'] * 2, ['rectifier'] * 2,
                    optimizer=optimizer, batch_size= batch_size)



In [ ]:
climin.initialize.randomize_normal(m.parameters.data, 0, 0.1)

In [ ]:
m.optimizer = 'adam'

In [ ]:
if os.path.isfile(sensor + '_' +('shore' if use_shore else 'surface') + '_parameters.npy'):
    m.parameters.data[:] = np.load(sensor + '_' +('shore' if use_shore else 'surface') + '_parameters.npy')
else:
    max_passes = 100
    max_iter = max_passes * X.shape[0] / batch_size
    n_report = X.shape[0] / batch_size

    stop = climin.stops.AfterNIterations(max_iter)
    pause = climin.stops.ModuloNIterations(n_report)

    for i, info in enumerate(m.powerfit((X,), (TX,), stop, pause)):
        print i, info['loss'], info['val_loss']

    np.save(sensor + '_' +('shore' if use_shore else 'surface') + '_parameters.npy', m.parameters.data.as_numpy_array())

In [ ]:
_f_latents = m.function(['inpt'], m.vae.recog.sample())
f_latents = lambda x: _f_latents(x.astype('float32'))

_f_mean = m.function(['inpt'], m.vae.recog.mean)
f_mean = lambda x: _f_mean(x.astype('float32'))

f_var = m.function(['inpt'], m.vae.recog.var)

In [ ]:
L = f_latents(TX.astype('float32'))
M = f_mean(TX.astype('float32'))
V = f_var(TX.astype('float32'))

### Matrix: uniform latent vs. feature

In [ ]:
def plot_latent_space(L,c):
    n = L.shape[1]
    fig, axs = plt.subplots(n, n, figsize=(10, 10))
    for i in range(n):
        for j in range(n):
            if i == j:
                axs[i,j].hist2d(c, L[:,i], bins=(50,50))
            elif i > j:
                axs[i,j].scatter(L[:,i], L[:,j], c=c, marker='o')
            else:
                axs[i,j].set_axis_off()
    plt.tight_layout()

In [ ]:
L.shape, TYp.shape

### Regression

In [ ]:
def fit_linearregression(x,y,tx,ty):
    m = linear_model.LinearRegression()
    m.fit(x, y)
    pred = m.predict(tx)
    error = np.sqrt(((pred-ty)**2).mean())
    return error, pred, m

In [ ]:
def fit_dt(x,y,tx,ty):
    import sklearn.tree
    m = sklearn.tree.DecisionTreeRegressor(max_depth=5)
    m.fit(x, y)
    pred = m.predict(tx)
    error = np.sqrt(((pred-ty)**2).mean())
    return error, pred, m

In [ ]:
def fit_mlp(x,y,tx,ty):
    x = x.astype('float32')
    y = y.astype('float32')
    tx = tx.astype('float32')
    ty = ty.astype('float32')
    m = learn_Mlp(x.shape[1], [64, 64], 1, 
            hidden_transfers=['rectifier', 'rectifier'], out_transfer='identity',
            loss=lambda x,y: (x-y)**2, 
            optimizer=optimizer, batch_size=batch_size, max_iter=max_iter)
    
    climin.initialize.randomize_normal(m.parameters.data, 0, 1e-2)
    
    stop = climin.stops.AfterNIterations(100)
    report = climin.stops.ModuloNIterations(10)

    for i, info in enumerate(m.powerfit((x.astype('float32'), y[:, np.newaxis].astype('float32')), (x[:1].astype('float32'), y[:, np.newaxis][:1].astype('float32')), stop, pause)):
        pass
    f_predict = m.function([m.inpt], m.output)
    pred = f_predict(tx.astype('float32'))[:,0]
    error = np.sqrt(((pred-ty)**2).mean())
    return error, pred, m

In [ ]:
regression_targets = [
            {'name': 'Force',
             'Y':train_surface['force'], 'TY':test_surface['force'],
            'visualY':train_surface['force'], 'visualTY':test_surface['force']},
            
            {'name': 'Pitch',
             'Y':Yp, 'TY':TYp,
             'visualY':Yp+np.random.uniform(size=Yp.shape)*unit_pitch-unit_pitch/2, 'visualTY':TYp+np.random.uniform(size=TYp.shape)*unit_pitch-unit_pitch/2},
            
            {'name': 'Roll',
             'Y':Yr, 'TY':TYr,
             'visualY':Yr+np.random.uniform(size=Yr.shape)*unit_roll-unit_roll/2, 'visualTY':TYr+np.random.uniform(size=TYr.shape)*unit_roll-unit_roll/2},
    
            {'name': 'Shore',
             'Y':Ys, 'TY':TYs,
             'visualY':Ys, 'visualTY':TYs},
           ]

In [ ]:
regression_algorithms = [
    {'name': 'Linear Regression',
     'fn': fit_linearregression
    },
    
    {'name': 'Decision Tree Regression',
     'fn': fit_dt
    },
    
    #{'name': 'MLP',
    # 'fn': fit_mlp
    #}
]

In [ ]:
theano.config.compute_test_value = 'off'

In [ ]:
fontsize = 16

In [ ]:
for alg in regression_algorithms:
    print '& \\multicolumn{2}{c}{' + alg['name'] + '} ',
    
print "\\\\\n",
    
for alg in regression_algorithms:
    print '& raw & latent',
    
print "\\\\\n",

for t in regression_targets:
    print t['name'],
    for alg in regression_algorithms:

        print '&',
        raw_error, raw_pred, raw_m = alg['fn'](X, t['Y'], TX, t['TY'])
        print "%.2f" % raw_error,
    
        print '&',
        latent_error, latent_pred, latent_m = alg['fn'](f_mean(X), t['Y'], f_mean(TX), t['TY'])
        print "%.2f" % latent_error,
        
        # plot latent value with highest correlation to physical value
        if alg['name'] == 'Linear Regression':
            L = f_latents(np.concatenate( (X, TX), 0))
            _m = linear_model.LinearRegression()
            _m.fit(L, np.concatenate( (t['Y'],t['TY']), 0) )
            indexes = np.argsort(-abs(_m.coef_))

            for index in indexes[:1]:
                fig, ax = plt.subplots(1,1, figsize=(4,4))
                ax.set_ylabel(t['name'], fontsize=fontsize)
                ax.set_xlabel('Latent variable #' + str(index) + ' [rel.]', fontsize=fontsize)

                ax.hexbin(L[:,index], np.concatenate( (t['visualY'], t['visualTY']), 0))
                ax.set_axis_bgcolor(plt.cm.jet(0))
                ax.set_xlim([-4,4])
                fig.savefig(t['name'] + '_latent_' + sensor + '.pdf')
                

        fig, ax = plt.subplots(1, 2, figsize=(2*4, 4))

        ax[0].hexbin(raw_pred, t['visualTY'])
        ax[0].set_xlim([np.min(t['visualTY']), np.max(t['visualTY'])])
        ax[0].plot([np.min(t['visualTY']),np.max(t['visualTY'])], [np.min(t['visualTY']), np.max(t['visualTY'])], c='r')
        ax[0].set_axis_bgcolor(plt.cm.jet(0))
        ax[0].set_title('raw taxel\nRMSE: ' + ("%.2f" % raw_error),  fontsize=fontsize)
        ax[0].set_ylabel(t['name'],  fontsize=fontsize)
        
        ax[1].hexbin(latent_pred, t['visualTY'])
        ax[1].set_xlim([np.min(t['visualTY']), np.max(t['visualTY'])])
        ax[1].plot([np.min(t['visualTY']),np.max(t['visualTY'])], [np.min(t['visualTY']), np.max(t['visualTY'])], c='r')
        ax[1].set_axis_bgcolor(plt.cm.jet(0))
        ax[1].set_title('latent\nRMSE: ' + ("%.2f" % latent_error),  fontsize=fontsize)
        ax[1].set_ylabel(t['name'],  fontsize=fontsize)
        fig.savefig(alg['name'].replace(' ', '') + '' + t['name'] + '_regression_' + sensor + '.pdf')
    print '\\\\\n',